In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PII. import Image

In [ ]:
%matplotlib inline #这条命令使matlibplot绘图时，直接在Notebook中显示
sys.path.append('..') #将上层目录导入进来，这样才可以执行下面的两条语句

from utils import label_map_util
from utils import visualization_utils as vis_util

## 导入包后，设置需要使用的模型：

In [ ]:
MODEL_NAME ='ssd_mobilenet_v1_coco_11_06_2017'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb' #'/frozen_inference_graph.pb'为后面需要导入的文件，里面包含了网络的结构和数据

PATH_TO_LABELS = os.path.join('data','mscoco_label_map.pbtxt') #'mscoco_label_map.pbtxt'文件保存了index到类别名的映射，该文件存在object_detection/data文件夹下面

NUM_CLASSES = 90  #os.path.join()连接两个文件名地址，比如：os.path.join("D:\","test.txt")，结果为:"D:\test.txt"

# 下载模型

In [ ]:
opener = urllib.request.URLopener() #打开链接
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE,MODEL_FILE) #将下载文件保存到指定文件
tar_file = tarfile.open(MODEL_FILE) #打开压缩文件
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name) #os.path.basename() 返回path最后的文件名
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file,os.getcwd()) #os.getcwd()返回当前目录；
        #tar_file.exract(file.directory) #将file取出放入当前目录

## 将模型导入计算图中

In [ ]:
detection_graph = tf.Graph() #tf.Graph()实例化了一个类：tensorflow中用于计算的数据流图
with detection_graph.as_default():
    old_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT,'rb') as fid:
        serialized_graph = fid.read()
        old_graph_def.ParseFromString(serialized_graph) #将模型解析到计算图中
        tf.import_graph_def(od_graph_def,name='') #将模型导入计算图中

## 下载label map

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map,max_num_classes=NUM_CLASSES,use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [ ]:
def load_image_into_numpy_array(image):
    (im_width,im_height) = image.size
    return np.array(image.getdata()).reshape((im_height,im_width,3)).astype(np.uint8)

## Detection

In [ ]:
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_iMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR,'image{}.jpg'.format(i)) for i in range(1,3)]
IMAGE_SIZE = (12,8) #output image size

In [ ]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0') #需要Input，input为image 
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')#输出该卷积层的output:检测框坐标
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')# 表示每个检测结果的confidence
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')# 表示每个框对应的类别
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')# 表示检测框的个数
        for image_path in TEST_IMAGE_PATHS:
            image = Image.open(image_path)
            image_np = load_image_into_numpy_array(image)
            image_np_expanded = np.expand_dims(image_np,axis=0)
            (boxes,scores,classes,num) = sess.run([detection_boxes,detection_scores,detection_classes,num_detections],feed_dict = {image_tensor:image_np_expanded})#用sess.run()进行计算
            #对得到的检测结果进行可视化
            vis_util.visualize_boxes_and_labels_on_image_array(image_np,np.squeeze(boxes),np.squeeze(classes).astype(np.int32),np.squeeze(scores),category_index,use_normalized_coordinates=True,line_thickness=8)#np.squeeze()删掉单维数组
            plt.figure(figsize=IMAGE_SIZE) # 画图之前首先设置figure对象，此函数相当于设置一块自定义大小的画布，使得后面的图形输出在这块规定了大小的画布上，其中参数figsize设置画布大小
            plt.imshow(image_np)